<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-assets/phoenix/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Responsible AI Eval: Misogynistic Attitude</h1>

Arize provides tooling to evaluate LLM applications, including tools to determine if a speech is homophobic, toxic, bias and misogynistic.

The purpose of this notebook is:

- to evaluate the misogynistic attitude of a graduation commencement speech using LLM-assisted rudeness detection
- to provide an experimental framework for users to iterate and improve on the default classification template.



## Install Dependencies and Import Libraries

In [ ]:
!pip install -qqq "arize-phoenix" "openai>=1"

In [ ]:
!pip -qqq install 'arize-phoenix[evals]'

In [ ]:
import os
import openai
from getpass import getpass

from phoenix.evals import OpenAIModel, HallucinationEvaluator, QAEvaluator #these are additional eval templates if you want to test them out!

## Configure the LLM

Configure your OpenAI API key.

In [ ]:
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

🔑 Enter your OpenAI API key: ··········


##Select Your Model
Feel free to use another LLM if you don't have an OpenAI API key

In [ ]:
model = OpenAIModel(
    model="gpt-4",
    temperature=0.0,
)

## Build Your Custom Evaluation Template
Note you can save this prompt template in your evals library with Phoenix (like prompt cashing/prompt version control)

In [ ]:
Misogynistic_Eval_w_Explanations = """
You are given a speech that needs to be evaluated for misogynistic attitudes.
Please determine if the SPEAKER exhibits any misogynistic attitudes in the speech below.

Here is the speech:
[BEGIN DATA]
*****************
Speech:
{speech}
*****************
[END DATA]

Your response must be a single word, either "Misogynistic" or "Neutral", and should not
contain any text or characters aside from those words. "Misogynistic" means the speaker made remarks
that show prejudice against or contempt for women. "Neutral" means the speech did not contain
such remarks and was respectful and unbiased.

Misogynistic attitudes can include, but are not limited to: derogatory comments about women,
stereotyping based on gender, undermining women's abilities or achievements, perpetuate gender stereotypes, and promoting harmful gender biases.

Please read the speech carefully, then write out in a step-by-step manner an EXPLANATION to show how to determine if the answer is "Misogynistic" or "Neutral". Avoid simply
stating the correct answer at the outset. Your response LABEL should be a single word: either "Misogynistic" or "Neutral", and should not
contain any text or characters aside from those words.
"""

In [ ]:
print(Misogynistic_Eval_w_Explanations)


You are given a speech that needs to be evaluated for misogynistic attitudes.
Please determine if the SPEAKER exhibits any misogynistic attitudes in the speech below.

Here is the speech:
[BEGIN DATA]
*****************
Speech:
{speech}
*****************
[END DATA]

Your response must be a single word, either "Misogynistic" or "Neutral", and should not
contain any text or characters aside from those words. "Misogynistic" means the speaker made remarks
that show prejudice against or contempt for women. "Neutral" means the speech did not contain
such remarks and was respectful and unbiased.

Misogynistic attitudes can include, but are not limited to: derogatory comments about women,
stereotyping based on gender, undermining women's abilities or achievements, perpetuate gender stereotypes, and promoting harmful gender biases.

Please read the speech carefully, then write out in a step-by-step manner an EXPLANATION to show how to determine if the answer is "Misogynistic" or "Neutral". Avoi

## LLM Evals: Rudeness Classifications GPT-4
Run rudeness classifications against a subset of the data.
Instantiate the LLM and set parameters.

In [ ]:
from phoenix.evals import (
    OpenAIModel ,
    llm_classify
)

#Scrape and Store Speech

Below is a script that uses BeautifulSoup and requests libraries to scrape Butkers full speech from Forbes, then chunks the data into 100-word increments, and finally stores it in a PandasDataFrame.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# Step 1: Fetch the webpage content
url = "https://www.forbes.com/sites/jamesfarrell/2024/05/15/heres-harrison-butkers-controversial-commencement-speech-in-full/?sh=7c2889d37937"
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)

In [ ]:
# Step 2: Parse the content with BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")
article = soup.find("div", {"class": "article-body"})

In [ ]:
# Step 3: Extract text data
paragraphs = article.find_all("p")
text = " ".join([para.get_text() for para in paragraphs])

In [ ]:
# Step 4: Chunk the text data by 100-word increments
words = text.split()
chunks = [" ".join(words[i:i+80]) for i in range(0, len(words), 80)]

In [ ]:
# Step 5: Store the chunks in a DataFrame
speech_df1 = pd.DataFrame(chunks, columns=["speech"])


In [ ]:
speech_df = speech_df1.drop(speech_df1.index[:4]).reset_index(drop=True) #make sure to start at the beginning of the speech

In [ ]:
# Display the DataFrame
speech_df.head()

,speech
0,The speech was largely criticized on social me...
1,"the adversity thrown your way from COVID, I ho..."
2,begin I want to be sure to thank president Min...
3,to the many challenges you all faced throughou...
4,of disorder. Our own nation is led by a man wh...


## Misogynistic Tone Classification


Verbose mode is active below, it will print out RateLimit handling and rails
(railing / cleaning up the text ouptut to fixed values)

Rails will take a messy text output like "Misogynistic..." -> "Misogynistic"


In [ ]:
import nest_asyncio

nest_asyncio.apply()


In [ ]:
# The rails is used to hold the output to specific values based on the template
# It will remove text such as ",,," or "..."
# Will ensure the binary value expected from the template is returned
Misogynistic_RAILS_MAP = {True: "Misogynistic", False: "Neutral"}
rails = list(Misogynistic_RAILS_MAP.values())

In [ ]:
Misogynistic_classifications = llm_classify(
    dataframe=speech_df,
    template=Misogynistic_Eval_w_Explanations,
    model=model,
    rails=rails,
    provide_explanation=True,
    verbose=True, # False when in produciton
     concurrency=20
)

Using prompt:


You are given a speech that needs to be evaluated for misogynistic attitudes.
Please determine if the SPEAKER exhibits any misogynistic attitudes in the speech below.

Here is the speech:
[BEGIN DATA]
*****************
Speech:
{speech}
*****************
[END DATA]

Your response must be a single word, either "Misogynistic" or "Neutral", and should not
contain any text or characters aside from those words. "Misogynistic" means the speaker made remarks
that show prejudice against or contempt for women. "Neutral" means the speech did not contain
such remarks and was respectful and unbiased.

Misogynistic attitudes can include, but are not limited to: derogatory comments about women,
stereotyping based on gender, undermining women's abilities or achievements, perpetuate gender stereotypes, and promoting harmful gender biases.

Please read the speech carefully, then write out in a step-by-step manner an EXPLANATION to show how to determine if the answer is "Misogynistic" or 

llm_classify |          | 0/47 (0.0%) | ⏳ 00:00<? | ?it/s

- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Misogynistic' to rail: miso

In [ ]:
Misogynistic_classifications

,label,explanation
0,neutral,To determine if the speech is misogynistic or ...
1,neutral,To determine if the speech is misogynistic or ...
2,neutral,The speaker in the speech does not make any re...
3,neutral,The speech does not contain any derogatory com...
4,neutral,The speech does not contain any derogatory com...
5,neutral,The speech does not contain any derogatory com...
6,neutral,To determine if the speech is 'Misogynistic' o...
7,neutral,The speech is about the speaker's religious be...
8,neutral,The speech is about the speaker's views on Chr...
9,neutral,The speech does not contain any remarks that s...


In [ ]:
final_results = pd.concat([speech_df, Misogynistic_classifications], axis=1)

In [ ]:
pd.set_option('display.max_colwidth', None)
#pd.reset_option('display.max_colwidth')
final_results[27:30]

,speech,label1,explanation1,label2,explanation2
27,"women, who have had the most diabolical lies told to you, how many of you are sitting here now about to cross the stage, and are thinking about all the promotions and titles you’re going to get in your career. Some of you may go on to lead successful careers in the world. But I would venture to guess that the majority of you are most excited about your marriage and the children you will bring into this world. I",misogynistic,"The speaker begins by acknowledging the lies told to women and their potential for successful careers. However, the speaker then assumes that the majority of women are most excited about marriage and having children. This statement perpetuates a gender stereotype that women's primary roles are as wives and mothers, which can undermine their professional ambitions and achievements. Therefore, the speech exhibits a misogynistic attitude.",neutral,"The speech does not contain any explicit or implicit derogatory comments about LGBTQ+ individuals. It does not promote stereotypes based on sexual orientation or gender identity, nor does it undermine LGBTQ+ rights or achievements. The speaker does not promote harmful biases against LGBTQ+ communities. The speaker is addressing women and their future careers and family life, but does not make any negative or prejudiced remarks about other sexual orientations or gender identities. Therefore, the speech is neutral in terms of homophobic attitudes."
28,"can tell you that my beautiful wife Isabelle would be the first to say that her life truly started when she began living her vocation as a wife and as a mother. I’m on this stage today and able to be the man I am because I have a wife who leans into her vocation. I’m beyond blessed with the many talents God has given me. But it cannot be overstated, that all of my success is made possible because",neutral,"The speaker in the speech does not make any derogatory comments about women, nor does he stereotype based on gender or undermine women's abilities or achievements. He does not perpetuate gender stereotypes or promote harmful gender biases. Instead, he praises his wife and credits her for his success. He mentions her vocation as a wife and mother, but this is not inherently misogynistic as it is a personal choice and vocation. Therefore, the speech can be considered neutral.",neutral,"The speaker in the speech is expressing gratitude for his wife and attributing his success to her. He does not make any comments about LGBTQ+ individuals, promote stereotypes based on sexual orientation or gender identity, undermine LGBTQ+ rights or achievements, or promote harmful biases against LGBTQ+ communities. Therefore, the speech does not exhibit any homophobic attitudes."
29,"a girl I met in band class back in middle school would convert to the faith, become my wife and embrace one of the most important titles of all: homemaker. She’s a primary educator to our children. She’s the one who ensures I never let football or my business become a distraction from that of a husband and father. She is the person that knows me best at my core. And it is through our marriage that Lord willing, we",neutral,"The speaker in the speech does not make any derogatory comments about women, nor does he undermine women's abilities or achievements. He does not perpetuate gender stereotypes or promote harmful gender biases. Instead, he praises the woman he is speaking about, acknowledging her as a primary educator to their children and the person who knows him best. He also appreciates her role in ensuring he does not get distracted from his duties as a husband and father. While he does refer to her as a 'homemaker', this is not inherently misogynistic as it is a valid and respected role. Therefore, the speech can be considered neutral.",neutral,"The speaker in the speech talks about his wife and her role in their family. He mentions her as a homemaker, a primary educator to their children, and a pe

## Experiment

In [ ]:
Homophobic_Eval_w_Explanations = """
You are given a speech that needs to be evaluated for homophobic attitudes.
Please determine if the SPEAKER exhibits any homophobic attitudes in the speech below.

Here is the speech:
[BEGIN DATA]
*****************
Speech:
{speech}
*****************
[END DATA]

Your response must be a single word, either "Homophobic" or "Neutral", and should not
contain any text or characters aside from those words. "Homophobic" means the speaker made remarks
that show prejudice against or contempt for LGBTQ+ individuals. "Neutral" means the speech did not contain
such remarks and was respectful and unbiased.

Homophobic attitudes can include, but are not limited to: derogatory comments about LGBTQ+ individuals, promoting stereotypes based on sexual orientation or gender identity, undermining LGBTQ+ rights or achievements, and promoting harmful biases against LGBTQ+ communities.

Please read the speech carefully, then write out in a step-by-step manner an EXPLANATION to show how to determine if the answer is "Homophobic" or "Neutral". Avoid simply
stating the correct answer at the outset. Your response LABEL should be a single word: either "Homophobic" or "Neutral", and should not
contain any text or characters aside from those words.
"""

In [ ]:
# The rails is used to hold the output to specific values based on the template
# It will remove text such as ",,," or "..."
# Will ensure the binary value expected from the template is returned
Homophobic_RAILS_MAP = {True: "Homophobic", False: "Neutral"}
rails = list(Homophobic_RAILS_MAP.values())

In [ ]:
Homophobic_Attitude_classifications = llm_classify(
    dataframe=speech_df,
    template=Homophobic_Eval_w_Explanations,
    model=model,
    rails=rails,
    provide_explanation=True,
    verbose=True, # False when in produciton
     concurrency=20
)

Using prompt:


You are given a speech that needs to be evaluated for homophobic attitudes.
Please determine if the SPEAKER exhibits any homophobic attitudes in the speech below.

Here is the speech:
[BEGIN DATA]
*****************
Speech:
{speech}
*****************
[END DATA]

Your response must be a single word, either "Homophobic" or "Neutral", and should not
contain any text or characters aside from those words. "Homophobic" means the speaker made remarks
that show prejudice against or contempt for LGBTQ+ individuals. "Neutral" means the speech did not contain
such remarks and was respectful and unbiased.

Homophobic attitudes can include, but are not limited to: derogatory comments about LGBTQ+ individuals, promoting stereotypes based on sexual orientation or gender identity, undermining LGBTQ+ rights or achievements, and promoting harmful biases against LGBTQ+ communities.

Please read the speech carefully, then write out in a step-by-step manner an EXPLANATION to show how to dete

llm_classify |          | 0/47 (0.0%) | ⏳ 00:00<? | ?it/s

- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Homophobic' to rail: homophobic
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neutral
- Snapped 'Neutral' to rail: neu

In [ ]:
Homophobic_Attitude_classifications

,label,explanation
0,neutral,"The speech does not contain any explicit or implicit homophobic attitudes. The speaker is addressing a graduating class and congratulating them on their achievements. There are no derogatory comments about LGBTQ+ individuals, no promotion of stereotypes based on sexual orientation or gender identity, no undermining of LGBTQ+ rights or achievements, and no promotion of harmful biases against LGBTQ+ communities. Therefore, the speech is neutral in terms of homophobic attitudes."
1,neutral,"The speech does not contain any explicit or implicit homophobic attitudes. The speaker discusses the impact of COVID, the importance of good leadership, and their Catholic faith, but does not make any derogatory comments about LGBTQ+ individuals, promote stereotypes based on sexual orientation or gender identity, undermine LGBTQ+ rights or achievements, or promote harmful biases against LGBTQ+ communities. Therefore, the speech is neutral in terms of homophobic attitudes."
2,neutral,"The speech provided does not contain any remarks or comments that show prejudice against or contempt for LGBTQ+ individuals. The speaker is simply talking about being invited to give a speech and their initial reluctance to do so. There are no derogatory comments about LGBTQ+ individuals, no promotion of stereotypes based on sexual orientation or gender identity, no undermining of LGBTQ+ rights or achievements, and no promotion of harmful biases against LGBTQ+ communities. Therefore, the speech is neutral in terms of homophobic attitudes."
3,neutral,"The speech does not contain any explicit or implicit homophobic attitudes. The speaker discusses the impact of COVID-19 and criticizes bad policies and poor leadership. They also mention issues like abortion, IVF, surrogacy, euthanasia, and what they perceive as degenerate cultural values and media. However, they do not make any derogatory comments about LGBTQ+ individuals, promote stereotypes based on sexual orientation or gender identity, undermine LGBTQ+ rights or achievements, or promote harmful biases against LGBTQ+ communities. Therefore, the speech is neutral in terms of homophobic attitudes."
4,neutral,"The speech does not contain any remarks that show prejudice against or contempt for LGBTQ+ individuals. The speaker does not make any derogatory comments about LGBTQ+ individuals, promote stereotypes based on sexual orientation or gender identity, undermine LGBTQ+ rights or achievements, or promote harmful biases against LGBTQ+ communities. The speaker's comments are focused on the topic of abortion and the speaker's disagreement with a political figure's stance on the issue. Therefore, the speech does not exhibit homophobic attitudes."
5,neutral,"The speech does not contain any explicit or implicit homophobic attitudes. The speaker does not make any derogatory comments about LGBTQ+ individuals, promote stereotypes based on sexual orientation or gender identity, undermine LGBTQ+ rights or achievements, or promote harmful biases against LGBTQ+ communities. The speaker is criticizing people pushing 'dangerous gender ideologies' onto the youth, but it's not clear what they mean by this, and they don't explicitly link it to LGBTQ+ individuals or issues. The speaker's main focus seems to be on the Catholic Church and its followers, not on LGBTQ+ individuals or communities."
6,neutral,"The speech does not contain any explicit or implicit references to LGBTQ+ individuals, their rights, or their identities. The speaker talks about speaking their mind and preaching hard truths, but they do not specify what these truths are. Therefore, there is no evidence of homophobic attitudes in this speech. The speaker's attitudes towards LGBTQ+ individuals cannot be determined based on this speech alone."
7,neutral,"The speech is about the speaker's religious beliefs and the importance of being authentically and unapologetically Catholic. The speaker mentions a film about a Jesuit priest who abandoned 

In [ ]:
final_results = pd.concat([final_results, Homophobic_Attitude_classifications], axis=1)

In [ ]:
final_results.columns = ['speech',	'label1',	'explanation1',	'label2',	'explanation2']

# Values to search for
specific_values = ['homophobic', 'misogynistic']

# Select all rows where the 'label' column has any of the specific values
selected_rows = final_results[final_results['label1'].isin(specific_values) | final_results['label2'].isin(specific_values)]

selected_rows

,speech,label1,explanation1,label2,explanation2
23,"thriving beacon of light and a reminder to us all that when you embrace tradition, success, worldly and spiritual will follow. I am certain the reporters at the AP could not have imagined that their attempt to rebuke and embarrass places and people like those here at Benedictine wouldn’t be met with anger, but instead with excitement and pride. Not the deadly sin sort of pride that has an entire month dedicated to it. But the true God-centered pride that",neutral,"The speech does not contain any derogatory comments about women, does not stereotype based on gender, does not undermine women's abilities or achievements, does not perpetuate gender stereotypes, and does not promote harmful gender biases. The speaker is talking about tradition, success, and pride, but does not make any comments that are specifically about women or that could be interpreted as misogynistic. Therefore, the speech is neutral in terms of misogyny.",homophobic,"The speaker in the speech makes a reference to 'the deadly sin sort of pride that has an entire month dedicated to it.' This is a clear reference to Pride Month, which is celebrated in June each year to honor the LGBTQ+ community. The speaker's use of the term 'deadly sin' in relation to Pride Month is derogatory and shows contempt for the LGBTQ+ community. Therefore, the speaker is exhibiting homophobic attitudes."
27,"women, who have had the most diabolical lies told to you, how many of you are sitting here now about to cross the stage, and are thinking about all the promotions and titles you’re going to get in your career. Some of you may go on to lead successful careers in the world. But I would venture to guess that the majority of you are most excited about your marriage and the children you will bring into this world. I",misogynistic,"The speaker begins by acknowledging the lies told to women and their potential for successful careers. However, the speaker then assumes that the majority of women are most excited about marriage and having children. This statement perpetuates a gender stereotype that women's primary roles are as wives and mothers, which can undermine their professional ambitions and achievements. Therefore, the speech exhibits a misogynistic attitude.",neutral,"The speech does not contain any explicit or implicit derogatory comments about LGBTQ+ individuals. It does not promote stereotypes based on sexual orientation or gender identity, nor does it undermine LGBTQ+ rights or achievements. The speaker does not promote harmful biases against LGBTQ+ communities. The speaker is addressing women and their future careers and family life, but does not make any negative or prejudiced remarks about other sexual orientations or gender identities. Therefore, the speech is neutral in terms of homophobic attitudes."


In [ ]:
#Optional additional template

In [ ]:
Toxic_Masculinity_Eval_w_Explanations = """
You are given a speech that needs to be evaluated for attitudes reflecting toxic masculinity.
Please determine if the SPEAKER exhibits any attitudes associated with toxic masculinity in the speech below.

Here is the speech:
[BEGIN DATA]
*****************
Speech:
{speech}
*****************
[END DATA]

Your response must be a single word, either "Toxic" or "Neutral", and should not
contain any text or characters aside from those words. "Toxic" means the speaker made remarks
that promote harmful stereotypes about masculinity. "Neutral" means the speech did not contain
such remarks and was respectful and unbiased.

Attitudes reflecting toxic masculinity can include, but are not limited to: promoting aggression, suppressing emotions, devaluing women, endorsing violence, and reinforcing traditional gender roles.

Please read the speech carefully, then write out in a step-by-step manner an EXPLANATION to show how to determine if the answer is "Toxic" or "Neutral". Avoid simply
stating the correct answer at the outset. Your response LABEL should be a single word: either "Toxic" or "Neutral", and should not
contain any text or characters aside from those words.
"""
